In [36]:
import pandas as pd
import numpy as np
import re

In [37]:
# Set Pandas display options to show all columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [38]:
# Step 1: Read the file without specifying headers
# file_path = r'D:\Gopal\commission model\Commision Python script as per shukat\Excel file use\Test File\Bajaj TW.xlsx'
# file_path = r'D:\Gopal\commission model\Commision Python script as per shukat\Excel file use\Test File\Chola Pvt car July24 .xlsx'
# file_path = r'D:\Gopal\commission model\Commision Python script as per shukat\Excel file use\Test File\Chola Tw july 2024.xlsx'
# file_path = r'D:\Gopal\commission model\Commision Python script as per shukat\Excel file use\Test File\chola cv july 24.xlsx'
# file_path = r'D:\Gopal\commission model\Commision Python script as per shukat\Excel file use\Test File\Digit 4w SATP July24.xlsx'
# file_path = r'D:\Gopal\commission model\Commision Python script as per shukat\Excel file use\Test File\Digit TW Comp + SATP + SAOD July 24.xlsx'
# file_path = r'D:\Gopal\commission model\Commision Python script as per shukat\Excel file use\Test File\ICICI Commerical July24.xlsx'
# file_path= r'D:\Gopal\commission model\Commision Python script as per shukat\Excel file use\Test File\ICICI TW new July24.xlsx'
# file_path = r'D:\Gopal\commission model\Commision Python script as per shukat\Excel file use\Test File\ICICI TW&PvtCar June24.xlsx'
file_path = r'D:\Gopal\commission model\Commision Python script as per shukat\Excel file use\Test File\New_India_June24.xlsx'
# file_path = r'D:\Gopal\commission model\Commision Python script as per shukat\Excel file use\Test File\United before April.xlsx'
# file_path = r'D:\Gopal\commission model\Commision Python script as per shukat\Excel file use\Test File\United_July24.xlsx'
# file_path = r'D:\Gopal\commission model\Commision Python script as per shukat\Excel file use\Test File\sbi_july.xlsx'
# file_path = r'D:\Gopal\commission model\Commision Python script as per shukat\Excel file use\Test File\SBI_June24.xlsx'
# file_path = r'D:\Gopal\commission model\Commision Python script as per shukat\Excel file use\Test File\Reliance CV 4 july 2024.xlsx' 
# file_path = r'D:\Gopal\commission model\Commision Python script as per shukat\Excel file use\Test File\Reliance TW July 24.xlsx'
# file_path = r'D:\Gopal\commission model\Commision Python script as per shukat\Excel file use\Test File\Reliance PVT TP 24 july 24.xlsx'
# file_path = r'D:\Gopal\commission model\Commision Python script as per shukat\Excel file use\Test File\TATA AIG 2w.xlsx'
# file_path = r'D:\Gopal\commission model\Commision Python script as per shukat\Excel file use\Test File\TATA AIG Pvtcar july 2024.xlsx'
# file_path = r'D:\Gopal\commission model\Commision Python script as per shukat\Excel file use\Test File\TATA AIG CV July 24.xlsx'
# file_path = r"D:\Gopal\commission model\Commision Python script as per shukat\Excel file use\Chola_TW data jan to july.xlsx"
# file_path = r'D:\Gopal\commission model\Commision Python script as per shukat\Excel file use\Test File\Digit 4W feb 24.xlsx'

df_initial = pd.read_excel(file_path)

In [39]:
df_initial.head(3)

,Product,OD Commission (including Rewards),TP Commission (including Rewards)
0,Private Car,20.0,15.0
1,Private Car Standalone OD,20.0,NaN
2,Private Car Standalone TP,NaN,15.0


In [40]:
# Function to find the first row that appears to be part of the table structure
def find_table_start(df):
    for idx, row in df.iterrows():
        if row.notnull().sum() >= 2 and not all(row.astype(str).str.contains('^Unnamed', na=False)):  # At least 2 non-null values and not all 'Unnamed'
            return idx
    return None

## to find the first numerical row 
def find_first_numerical_row(df):
    for idx, row in df.iterrows():
        # Check if any value in the row is numerical (int, float) or a digit-like string
        if row.apply(lambda x: not pd.isna(x) and (isinstance(x, (int, float)) or (isinstance(x, str) and (x.isdigit() or (x.endswith('%')))))).any():
            return idx
    return None

# to find the name of the column from which the numerical value start
# def find_row_with_most_numerical_values(df):
#     max_count = 0
#     row_with_most_numericals = None
#     first_numerical_column_name = None
    
#     for idx, row in df.iterrows():
#         # Check if any value in the row is numerical (int, float) or a digit-like string
#         is_numerical = row.apply(lambda x: not pd.isna(x) and (isinstance(x, (int, float)) or (isinstance(x, str) and x.isdigit())))
        
#         # Count the number of numerical values in the row
#         numerical_count = is_numerical.sum()
        
#         if numerical_count > max_count:
#             max_count = numerical_count
#             row_with_most_numericals = idx
            
#             # Find the first column name with a numerical value in this row
#             first_numerical_column = row.index[is_numerical]
#             if len(first_numerical_column) > 0:
#                 first_numerical_column_name = first_numerical_column[0]
#             else:
#                 first_numerical_column_name = None
    
#     return row_with_most_numericals, first_numerical_column_name

def extract_numerical_value(value):
    # Regular expression to match numeric values, optional space after '-', and optional '%' at the end
    pattern = re.compile(r'-\s*\d+(\.\d+)?%?$|^\d+(\.\d+)?%?$')
    match = pattern.search(str(value))
    return match.group() if match else None

def find_row_with_most_numerical_values(df):
    max_count = 0
    row_with_most_numericals = None
    first_numerical_column_name = None

    for idx, row in df.iterrows():
        # Extract numerical parts from each cell if present
        is_numerical = row.apply(lambda x: not pd.isna(x) and (
            isinstance(x, (int, float)) or
            extract_numerical_value(x) is not None
        ))
        
        # Count the number of numerical values in the row
        numerical_count = is_numerical.sum()
        
        if numerical_count > max_count:
            max_count = numerical_count
            row_with_most_numericals = idx
            
            # Find the first column name with a numerical value in this row
            first_numerical_column = row.index[is_numerical]
            # print(first_numerical_column)
            if len(first_numerical_column) > 0:
                if first_numerical_column[0] == df.columns[0]:
                    first_numerical_column_name = first_numerical_column[1] if len(first_numerical_column) > 1 else None
                else:
                    first_numerical_column_name = first_numerical_column[0]
                # print(first_numerical_column_name)
            else:
                first_numerical_column_name = None
    
    return row_with_most_numericals, first_numerical_column_name


# def find_all_numerical_columns(df):
#     numerical_columns = []

#     # Iterate over each column
#     for col in df.columns:
#         # Identify the values in the column that match the condition
#         matching_mask = df[col].apply(
#             lambda x: not pd.isna(x) and (
#                 isinstance(x, (int, float)) or
#                 (isinstance(x, str) and (x.isdigit() or (x.endswith('%'))))
#             )
#         )
#         if matching_mask.any():
#             numerical_columns.append(col)
    
#     # Return the first numerical column if any are found, along with the list of all numerical columns
#     if numerical_columns:
#         first_numerical_column = numerical_columns[0]
#         return first_numerical_column, numerical_columns
#     else:
#         return None, []

def clean_text(x):
    if isinstance(x, str):
        # Replace newline characters with a space
        x = x.replace('\n', ' ').replace('\r', ' ')
        # Remove extra spaces (multiple spaces replaced by a single space)
        x = ' '.join(x.split())
    return x

def clean_column_headers(df):
    new_columns = []
    for col in df.columns:
        if col.startswith('Unnamed'):
            # Strip 'Unnamed' prefix and leading/trailing spaces
            new_col = col.split('__', 1)[-1].strip()
        else:
            new_col = col
        new_columns.append(new_col)
    df.columns = new_columns
    return df
    

# Function to clean the text inside parentheses
def clean_parentheses_content(match):
    content = match.group(1)
    # cleaned_content = re.sub(r'[^\w\s]', '', content)  # Remove unwanted symbols
    cleaned_content = re.sub(r'\*', '', content)
    cleaned_content = cleaned_content.strip()  # Remove leading and trailing spaces
    return f"_{cleaned_content}"

# Function to process text
def replace_brackets(text):
    if isinstance(text, str):
        # Replace parentheses and remove unwanted symbols within them
        text = re.sub(r'\((.*?)\)', clean_parentheses_content, text)
        # Remove extra spaces around underscores
        text = re.sub(r'\s*_\s*', '_', text)
    return text


# Function to find the common prefix among a list of strings
def common_prefix(strings):
    if not strings:
        return ""
    # Start with the first string in the list as the common prefix
    prefix = strings[0]
    for s in strings[1:]:
        # Compare the current prefix with each string
        while s[:len(prefix)] != prefix and prefix:
            # Reduce the prefix by one character at a time
            prefix = prefix[:-1]
    return prefix

# Function to rename columns with condition
def rename_columns(df):
    cols = list(df.columns)
    # Exclude columns that start with the exclude_prefix from prefix comparison
    cols_to_check = [col for col in cols if not col.startswith('variable')]
    
    common_prefix_str = common_prefix(cols_to_check)
    
    new_cols = []
    for col in cols:
        if col.startswith('variable'):
            new_cols.append(col)
        elif col.startswith(common_prefix_str):
            new_cols.append(col[len(common_prefix_str):])
        else:
            new_cols.append
    # print(new_cols)
    return new_cols


def strip_spaces(text):
    if isinstance(text, str):
        return text.strip()
    return text


def find_last_numerical_row(df, exclude_columns=[]):
    for idx, row in df.iloc[::-1].iterrows():
        # Filter out the excluded columns
        filtered_row = row.drop(labels=exclude_columns)
        # Check if any value in the filtered row is numerical (int, float) or a digit-like string
        if filtered_row.apply(lambda x: not pd.isna(x) and (isinstance(x, (int, float)) or (isinstance(x, str) and (x.isdigit() or x.endswith('%'))))).any():
            return idx
    return None



# def find_all_prefixes(columns):
#     exclude_prefixes = 'variable'
#     prefixes = set()
#     for col in columns:
#         if any(col.startswith(exclude) for exclude in exclude_prefixes):
#             continue
#         parts = col.split('_')
#         for i in range(1, len(parts)):
#             prefix = '_'.join(parts[:i]) + '_'
#             prefixes.add(prefix)
#     return prefixes

# def remove_prefixes(df, prefixes):
#     # exclude_prefixes = 'variable'
#     new_columns = []
#     for col in df.columns:
#         new_col = col
#         for prefix in prefixes:
#             new_col = new_col.replace(prefix, '')
#         if new_col.startswith('_'):
#             new_col = new_col.lstrip('_')
#         new_columns.append(new_col)
#     df.columns = new_columns
#     # print(new_columns)
#     return df


def find_all_prefixes(columns, exclude_prefixes=['variable']):
    prefixes = set()
    for col in columns:
        if any(col.startswith(exclude) for exclude in exclude_prefixes):
            continue
        parts = col.split('__')
        for i in range(1, len(parts)):
            prefix = '__'.join(parts[:i]) + '__'
            prefixes.add(prefix)
    return prefixes

def remove_prefixes(df, prefixes):
    new_columns = []
    for col in df.columns:
        new_col = col
        for prefix in sorted(prefixes, key=len, reverse=True):  # Sort by length to avoid partial replacements
            if new_col.startswith(prefix):
                new_col = new_col[len(prefix):]
                break  # Only remove the longest matching prefix
        new_columns.append(new_col)
    df.columns = new_columns
    return df


def find_last_numerical_row_fordrop(df):
    for idx, row in df.iloc[::-1].iterrows():
        # Check if any value in the row is numerical (int, float) or a digit-like string
        if row.apply(lambda x: not pd.isna(x) and (isinstance(x, (int, float)) or (isinstance(x, str) and (x.isdigit() or x.endswith('%'))))).any():
            return idx
    return None

In [41]:
# Step 2: Check the first numerical row index of df_initial before converting and preprocess the data
first_num_row_condition = find_first_numerical_row(df_initial) 

if first_num_row_condition > 0:
    # Step 2: Check if all initial headers are 'Unnamed'
    if all(df_initial.columns.astype(str).str.contains('^Unnamed', na=False)):
        # All initial headers are 'Unnamed', find the table start row using df_initial
        table_start_idx = find_table_start(df_initial)
        
        # If table_start_idx is None, there is no valid table structure in the file
        if table_start_idx is None:
            raise ValueError("No valid table structure found in the file.")
        
        # Adjust df_initial to start from the detected table start row
        df = df_initial.iloc[table_start_idx + 1:].reset_index(drop=True)

        # print(df)
        df.columns = df_initial.iloc[table_start_idx]


    

        # # Get the current column names
        columns = df.columns

        # # Create a list to store the new column names with forward fill
        new_columns = [columns[0]]  # Start with the first column name

        # Iterate through the columns and forward fill wherer the column name is null or nan for this case only
        for col in columns[1:]:
            if pd.isna(col):
                new_columns.append(new_columns[-1])  # Use the last valid column name
            else:
                new_columns.append(col)

        # # Assign the new column names to the DataFrame
        df.columns = new_columns       

        #         # Step 2: Initialize a list to store notes
        # notes = []
        # #print(notes)

        # # Step 3: Iterate through each row in the DataFrame to separate notes and remove them from df
        # index_to_drop = []  # To store indices of rows to drop from df
        # # for index, row in df.iterrows():
        # #     if pd.notna(row[0]) and not any(row.dropna().index[1:]):  # Check if it's a note row
        # #         notes.append(row[0])  # Append note to notes list
        # #         index_to_drop.append(index)  # Mark row index to drop

        # for index, row in df.iterrows():
        #     if (pd.notna(row[0]) or pd.notna(row[1])) and not any(row.dropna().index[2:]):  # Check if it's a note row
        #         notes.append(row[0] if pd.notna(row[0]) else row[1])  # Append note to notes list
        #         index_to_drop.append(index) 

        # # Step 4: Keep rows up to and including the smallest index in index_to_drop
        # if index_to_drop:
        #     min_index = min(index_to_drop)  # Find the smallest index to keep
        #     df = df.iloc[:min_index].reset_index(drop=True)  # Keep rows from the beginning up to but not including this index

        # # Step 2: Drop rows with all NaN values
        df = df.dropna(how='all').reset_index(drop=True)

        # # Drop columns where all values are NaN
        df = df.dropna(axis=1, how='all')

        # Step 3: Find the first numerical row
        first_num_row_idx = find_first_numerical_row(df)

        last_numerical_idx = find_last_numerical_row(df)



        if last_numerical_idx is not None:
                df = df.iloc[:last_numerical_idx + 1]

        #         # # Step 2: Drop rows with all NaN values
        # df = df.dropna(how='all').reset_index(drop=True)

        # # # Drop columns where all values are NaN
        # df = df.dropna(axis=1, how='all')

            # Process only if the first numerical row is not the first row of the DataFrame
        if first_num_row_idx > 0:
            # Step 4: Extract original headers and rows above the first numerical row
            original_headers = df.columns.tolist()
            above_rows = df.iloc[:first_num_row_idx]

            ## for the Filling null value in row by last valide value to get proper header 
            above_rows_filled = above_rows.fillna(method='ffill', axis=1)

            ## Assign the column header propely to use for the further work
            df.iloc[:first_num_row_idx] = above_rows_filled

            # Collect all values from above rows into a single list
            above_rows_values = above_rows.values.flatten().tolist()

            # Create combined headers by appending values above to the original headers
            # combined_headers = [f"{str(header)}_{str(value)}" if not pd.isna(value) else header for header, value in zip(original_headers, above_rows_values)]
            
            ## below code to  handle the  if single or multiple row under the above rows to create proper Header of columns
            combined_headers = original_headers[:]  # Start with original headers
            for idx, row in above_rows.iterrows():
                combined_headers = [
                    f"{header}__{str(value)}" if not pd.isna(value) else header
                    for header, value in zip(combined_headers, row)
                ]

            # Create new DataFrame with combined headers
            new_df = pd.DataFrame(df.iloc[first_num_row_idx:].values, columns=combined_headers)

            new_df = new_df.applymap(clean_text) ## for multiline text or wrap text in row or cell

            # Apply the cleaning function to the column names
            new_df.columns = [clean_text(col) for col in new_df.columns] ## removing text of multiline and wrap from heading

            # Replace NaN column headers with 'type'
            new_df.columns = ['Policy_type' if pd.isna(col) else col for col in new_df.columns]

            new_df = clean_column_headers(new_df)

            # Call the function
            first_numerical_row_idx, columns_with_numerical_values = find_row_with_most_numerical_values(new_df)
            # columns_with_numerical_values, all_numerical_columns = find_all_numerical_columns(new_df)
            
            col_idx = new_df.columns.get_loc(columns_with_numerical_values)
            columns_before = new_df.columns[:col_idx]

            # Step 1: Identify duplicated columns
            duplicate_columns = []

            for col in columns_before:
                if col in new_df.columns:
                    for other_col in new_df.columns:
                        if col != other_col and new_df[col].equals(new_df[other_col]):
                            duplicate_columns.append(other_col)

            # Step 2: Drop duplicate columns
            new_df.drop(columns=duplicate_columns, inplace=True)

            new_df[columns_before] = new_df[columns_before].fillna(method='ffill')

            last_numerical_row = find_last_numerical_row(new_df, columns_before)

            

            for col in new_df.columns[col_idx:]:
                new_df[col] = new_df[col].apply(lambda x: f"{float(x) * 100:.2f}" if isinstance(x, (int, float)) and str(x).startswith('0.') else x)  #% symbol removing and value at time of read convert to decimal reconvert to same


            # new_df = new_df.applymap(clean_text) ## for multiline text or wrap text in row or cell

            # # Apply the cleaning function to the column names
            # new_df.columns = [clean_text(col) for col in new_df.columns] ## removing text of multiline and wrap from heading

                # Loop through the columns and rename if they start with 'All_Broker_'
            new_columns = {col: col.split('__')[0] for col in new_df.columns if col.lower().startswith('all broker -')}

            # Rename the columns in the DataFrame
            new_df.rename(columns=new_columns, inplace=True)

            # Melt the DataFrame
            value_vars = new_df.columns[col_idx:]

            melted_df = pd.melt(new_df, id_vars=list(columns_before), value_vars=value_vars, var_name='variable', value_name='commission')

            # Split the 'variable' column based on '/' and '_' and expand into separate columns
            # expanded_cols = melted_df['variable'].str.split(r'[_]', expand=True)
            expanded_cols = melted_df['variable'].str.split(r'__', expand=True)
            
            # Rename columns based on the maximum number of splits
            expanded_cols.columns = [f'variable_{i+1}' for i in range(expanded_cols.shape[1])]
            
            # Concatenate the expanded columns to the melted DataFrame
            melted_df = pd.concat([melted_df, expanded_cols], axis=1)
            
            # Rearrange columns: expanded columns after id_vars and before 'commission'
            id_vars = list(columns_before)
            value_name = 'commission'
            
            columns_order = id_vars + [col for col in expanded_cols.columns] + [value_name]
            melted_df = melted_df[columns_order]


                        # Remove rows where the commission value is a substring of any value in the id_vars columns
            rows_to_drop = melted_df.apply(
                lambda row: any(str(row[value_name]) in str(row[id_var]) for id_var in id_vars), axis=1)

            melted_df = melted_df[~rows_to_drop].reset_index(drop=True)

            
            melted_df.columns = [col.split('__')[-1] if col.lower().startswith('all broker') else col for col in melted_df.columns]
            melted_df= melted_df.dropna(subset=['commission']).reset_index(drop=True)


            melted_df.columns = rename_columns(melted_df)
            # Apply the function to the entire DataFrame
            melted_df = melted_df.applymap(replace_brackets)

            # # Remove rows where any value in the 'commission' column matches any value in the 'id_vars' columns
            # rows_to_drop = melted_df.apply(
            #     lambda row: row[id_vars].isin([row[value_name]]).any(), axis=1)

            # melted_df = melted_df[~rows_to_drop].reset_index(drop=True)

            # Apply the function to the entire DataFrame
            melted_df = melted_df.applymap(strip_spaces)


            # Find all possible prefixes excluding certain prefixes
            all_prefixes = find_all_prefixes(melted_df.columns)
            # Remove all identified prefixes
            melted_df = remove_prefixes(melted_df, all_prefixes)

        
        else:
            # Step 2: Drop rows with all NaN values
            df = df.dropna(how='all').reset_index(drop=True)

            # Drop columns where all values are NaN
            df = df.dropna(axis=1, how='all')

            # # Step 2: Initialize a list to store notes
            # notes = []
            # #print(notes)

            # # Step 3: Iterate through each row in the DataFrame to separate notes and remove them from df
            # index_to_drop = []  # To store indices of rows to drop from df
            # # for index, row in df.iterrows():
            # #     if pd.notna(row[0]) and not any(row.dropna().index[1:]):  # Check if it's a note row
            # #         notes.append(row[0])  # Append note to notes list
            # #         index_to_drop.append(index)  # Mark row index to drop

            # for index, row in df.iterrows():
            #     if (pd.notna(row[0]) or pd.notna(row[1])) and not any(row.dropna().index[2:]):  # Check if it's a note row
            #         notes.append(row[0] if pd.notna(row[0]) else row[1])  # Append note to notes list
            #         index_to_drop.append(index) 

            # # Step 4: Keep rows up to and including the smallest index in index_to_drop
            # if index_to_drop:
            #     min_index = min(index_to_drop)  # Find the smallest index to keep
            #     df = df.iloc[:min_index].reset_index(drop=True)  # Keep rows from the beginning up to but not including this index


            new_df = df.copy()

            new_df = new_df.applymap(clean_text) ## for multiline text or wrap text in row or cell

            # Apply the cleaning function to the column names
            new_df.columns = [clean_text(col) for col in new_df.columns] ## removing text of multiline and wrap from heading


            new_df = clean_column_headers(new_df)

            # Call the function
            first_numerical_row_idx, columns_with_numerical_values = find_row_with_most_numerical_values(new_df)
            # columns_with_numerical_values, all_numerical_columns = find_all_numerical_columns(new_df)
            
            col_idx = new_df.columns.get_loc(columns_with_numerical_values)
            columns_before = new_df.columns[:col_idx]

            new_df[columns_before] = new_df[columns_before].fillna(method='ffill')

            last_numerical_row = find_last_numerical_row(new_df, columns_before)

            for col in new_df.columns[col_idx:]:
                new_df[col] = new_df[col].apply(lambda x: f"{float(x) * 100:.2f}" if isinstance(x, (int, float)) and str(x).startswith('0.') else x)  #% symbol removing and value at time of read convert to decimal reconvert to same


            # new_df = new_df.applymap(clean_text) ## for multiline text or wrap text in row or cell

            # # Apply the cleaning function to the column names
            # new_df.columns = [clean_text(col) for col in new_df.columns] ## removing text of multiline and wrap from heading


                # Loop through the columns and rename if they start with 'All_Broker_'
            new_columns = {col: col.split('__')[0] for col in df.columns if col.lower().startswith('all broker -')}

            # Rename the columns in the DataFrame
            new_df.rename(columns=new_columns, inplace=True)

            # Melt the DataFrame
            value_vars = new_df.columns[col_idx:]
            melted_df = pd.melt(new_df, id_vars=list(columns_before), value_vars=value_vars, var_name='variable', value_name='commission')

            # Split the 'variable' column based on '/' and '_' and expand into separate columns
            # expanded_cols = melted_df['variable'].str.split(r'[_]', expand=True)
            expanded_cols = melted_df['variable'].str.split(r'__', expand=True)
            
            # Rename columns based on the maximum number of splits
            expanded_cols.columns = [f'variable_{i+1}' for i in range(expanded_cols.shape[1])]
            
            # Concatenate the expanded columns to the melted DataFrame
            melted_df = pd.concat([melted_df, expanded_cols], axis=1)
            
            # Rearrange columns: expanded columns after id_vars and before 'commission'
            id_vars = list(columns_before)
            value_name = 'commission'
            
            columns_order = id_vars + [col for col in expanded_cols.columns] + [value_name]
            melted_df = melted_df[columns_order]


                        # Remove rows where the commission value is a substring of any value in the id_vars columns
            rows_to_drop = melted_df.apply(
                lambda row: any(str(row[value_name]) in str(row[id_var]) for id_var in id_vars), axis=1)

            melted_df = melted_df[~rows_to_drop].reset_index(drop=True)

            melted_df.columns = [col.split('__')[-1] if col.lower().startswith('all broker') else col for col in melted_df.columns]
            melted_df= melted_df.dropna(subset=['commission']).reset_index(drop=True) 

            melted_df.columns = rename_columns(melted_df) 
            # Apply the function to the entire DataFrame
            melted_df = melted_df.applymap(replace_brackets)

            # Apply the function to the entire DataFrame
            melted_df = melted_df.applymap(strip_spaces)

            # Find all possible prefixes excluding certain prefixes
            all_prefixes = find_all_prefixes(melted_df.columns)
            # Remove all identified prefixes
            melted_df = remove_prefixes(melted_df, all_prefixes)

    else:
        df =  df_initial.copy()

        # Step 1: Replace null or unnamed headers with inferred names (if needed)
        unnamed_columns = df.columns.str.contains('^Unnamed')
        new_columns = []
        last_valid_header = None

        for i, col in enumerate(df.columns):
            if unnamed_columns[i]:
                if last_valid_header is not None:
                    new_columns.append(last_valid_header)
                else:
                    new_columns.append('Unnamed')
            else:
                new_columns.append(col)
                last_valid_header = col

        df.columns = new_columns

        # Step 2: Drop rows with all NaN values as well as column also with complete NaN value
        df = df.dropna(how='all').reset_index(drop=True) # for row drop
        df = df.dropna(axis=1, how='all') # for Column drop

        # # Step 2: Initialize a list to store notes
        # notes = []
        # #print(notes)

        # # Step 3: Iterate through each row in the DataFrame to separate notes and remove them from df
        # index_to_drop = []  # To store indices of rows to drop from df
        # # for index, row in df.iterrows():
        # #     if pd.notna(row[0]) and not any(row.dropna().index[1:]):  # Check if it's a note row
        # #         notes.append(row[0])  # Append note to notes list
        # #         index_to_drop.append(index)  # Mark row index to drop

        # for index, row in df.iterrows():
        #     if (pd.notna(row[0]) or pd.notna(row[1])) and not any(row.dropna().index[2:]):  # Check if it's a note row
        #         notes.append(row[0] if pd.notna(row[0]) else row[1])  # Append note to notes list
        #         index_to_drop.append(index) 

        # # Step 4: Keep rows up to and including the smallest index in index_to_drop
        # if index_to_drop:
        #     min_index = min(index_to_drop)  # Find the smallest index to keep
        #     df = df.iloc[:min_index].reset_index(drop=True)  # Keep rows from the beginning up to but not including this index

        # Step 3: Find the first numerical row
        first_num_row_idx = find_first_numerical_row(df)

        # Process only if the first numerical row is not the first row of the DataFrame
        if first_num_row_idx > 0:
            # Step 4: Extract original headers and rows above the first numerical row
            original_headers = df.columns.tolist()
            above_rows = df.iloc[:first_num_row_idx]

            # to fill the row horizontaly to make proper header    
            above_rows_filled = above_rows.fillna(method='ffill', axis=1)

            # assigning the value find in above step to make proper header
            df.iloc[:first_num_row_idx] = above_rows_filled

            # Collect all values from above rows into a single list
            above_rows_values = above_rows.values.flatten().tolist()

            # Create combined headers by appending values above to the original headers
            # combined_headers = [f"{str(header)}_{str(value)}" if not pd.isna(value) else header for header, value in zip(original_headers, above_rows_values)]

            ## the Below code is useful to handle if there is more than one row in the above row to proper fill
            combined_headers = original_headers[:]  # Start with original headers
            for idx, row in above_rows.iterrows():
                combined_headers = [
                    f"{header}__{str(value)}" if not pd.isna(value) else header
                    for header, value in zip(combined_headers, row)
                ]

            # Create new DataFrame with combined headers
            new_df = pd.DataFrame(df.iloc[first_num_row_idx:].values, columns=combined_headers)

            new_df = new_df.applymap(clean_text) ## for multiline text or wrap text in row or cell

            # Apply the cleaning function to the column names
            new_df.columns = [clean_text(col) for col in new_df.columns] ## removing text of multiline and wrap from heading

            new_df = clean_column_headers(new_df)

            # Call the function to find the first numerical column to use melted df
            first_numerical_row_idx, columns_with_numerical_values = find_row_with_most_numerical_values(new_df)
            # columns_with_numerical_values, all_numerical_columns = find_all_numerical_columns(new_df)
                
            col_idx = new_df.columns.get_loc(columns_with_numerical_values)
            columns_before = new_df.columns[:col_idx]

            new_df[columns_before] = new_df[columns_before].fillna(method='ffill')

            last_numerical_row = find_last_numerical_row(new_df, columns_before)

            for col in new_df.columns[col_idx:]:
                new_df[col] = new_df[col].apply(lambda x: f"{float(x) * 100:.2f}" if isinstance(x, (int, float)) and str(x).startswith('0.') else x)  #% symbol removing and value at time of read convert to decimal reconvert to same


            # new_df = new_df.applymap(clean_text) ## for multiline text or wrap text in row or cell

            #     # Apply the cleaning function to the column names
            # new_df.columns = [clean_text(col) for col in new_df.columns] ## removing text of multiline and wrap from heading

                # Loop through the columns and rename if they start with 'All_Broker_'
            new_columns = {col: col.split('__')[0] for col in df.columns if col.lower().startswith('all broker -')}

            # Rename the columns in the DataFrame
            new_df.rename(columns=new_columns, inplace=True)

                # Melt the DataFrame
            value_vars = new_df.columns[col_idx:]
            melted_df = pd.melt(new_df, id_vars=list(columns_before), value_vars=value_vars, var_name='variable', value_name='commission')

                # Split the 'variable' column based on '/' and '_' and expand into separate columns
            # expanded_cols = melted_df['variable'].str.split(r'[_]', expand=True)
            expanded_cols = melted_df['variable'].str.split(r'__', expand=True)
                
                # Rename columns based on the maximum number of splits
            expanded_cols.columns = [f'variable_{i+1}' for i in range(expanded_cols.shape[1])]
                
                # Concatenate the expanded columns to the melted DataFrame
            melted_df = pd.concat([melted_df, expanded_cols], axis=1)
                
                # Rearrange columns: expanded columns after id_vars and before 'commission'
            id_vars = list(columns_before)
            value_name = 'commission'
                
            columns_order = id_vars + [col for col in expanded_cols.columns] + [value_name]
            melted_df = melted_df[columns_order]

                        # Remove rows where the commission value is a substring of any value in the id_vars columns
            rows_to_drop = melted_df.apply(
                lambda row: any(str(row[value_name]) in str(row[id_var]) for id_var in id_vars), axis=1)

            melted_df = melted_df[~rows_to_drop].reset_index(drop=True)

            melted_df.columns = [col.split('__')[-1] if col.lower().startswith('all broker') else col for col in melted_df.columns]
            melted_df= melted_df.dropna(subset=['commission']).reset_index(drop=True)   
            
            melted_df.columns = rename_columns(melted_df)
            # Apply the function to the entire DataFrame
            melted_df = melted_df.applymap(replace_brackets)

            # Apply the function to the entire DataFrame
            melted_df = melted_df.applymap(strip_spaces)

            # Find all possible prefixes excluding certain prefixes
            all_prefixes = find_all_prefixes(melted_df.columns)
            # Remove all identified prefixes
            melted_df = remove_prefixes(melted_df, all_prefixes)

        else:
            # Step 2: Drop rows with all NaN values
            df = df.dropna(how='all').reset_index(drop=True)

            # Drop columns where all values are NaN
            df = df.dropna(axis=1, how='all')

            # # Step 2: Initialize a list to store notes
            # notes = []
            # #print(notes)

            # # Step 3: Iterate through each row in the DataFrame to separate notes and remove them from df
            # index_to_drop = []  # To store indices of rows to drop from df
            # # for index, row in df.iterrows():
            # #     if pd.notna(row[0]) and not any(row.dropna().index[1:]):  # Check if it's a note row
            # #         notes.append(row[0])  # Append note to notes list
            # #         index_to_drop.append(index)  # Mark row index to drop


            # for index, row in df.iterrows():
            #     if (pd.notna(row[0]) or pd.notna(row[1])) and not any(row.dropna().index[2:]):  # Check if it's a note row
            #         notes.append(row[0] if pd.notna(row[0]) else row[1])  # Append note to notes list
            #         index_to_drop.append(index) 

            # # Step 4: Keep rows up to and including the smallest index in index_to_drop
            # if index_to_drop:
            #     min_index = min(index_to_drop)  # Find the smallest index to keep
            #     df = df.iloc[:min_index].reset_index(drop=True)  # Keep rows from the beginning up to but not including this index

            new_df = df.copy()

            new_df = new_df.applymap(clean_text) ## for multiline text or wrap text in row or cell

            # Apply the cleaning function to the column names
            new_df.columns = [clean_text(col) for col in new_df.columns] ## removing text of multiline and wrap from heading

            new_df = clean_column_headers(new_df)

            # Call the function
            first_numerical_row_idx, columns_with_numerical_values = find_row_with_most_numerical_values(new_df)
            # columns_with_numerical_values, all_numerical_columns = find_all_numerical_columns(new_df)
            
            col_idx = new_df.columns.get_loc(columns_with_numerical_values)
            columns_before = new_df.columns[:col_idx]

            new_df[columns_before] = new_df[columns_before].fillna(method='ffill')

            last_numerical_row = find_last_numerical_row(new_df, columns_before)

            for col in new_df.columns[col_idx:]:
                new_df[col] = new_df[col].apply(lambda x: f"{float(x) * 100:.2f}" if isinstance(x, (int, float)) and str(x).startswith('0.') else x)  #% symbol removing and value at time of read convert to decimal reconvert to same


            # new_df = new_df.applymap(clean_text) ## for multiline text or wrap text in row or cell

            # # Apply the cleaning function to the column names
            # new_df.columns = [clean_text(col) for col in new_df.columns] ## removing text of multiline and wrap from heading

            # Loop through the columns and rename if they start with 'All_Broker_'
            new_columns = {col: col.split('__')[0] for col in df.columns if col.lower().startswith('all broker -')}


            # Rename the columns in the DataFrame
            new_df.rename(columns=new_columns, inplace=True)

            # Melt the DataFrame
            value_vars = new_df.columns[col_idx:]
            melted_df = pd.melt(new_df, id_vars=list(columns_before), value_vars=value_vars, var_name='variable', value_name='commission')

            # Split the 'variable' column based on '/' and '_' and expand into separate columns
            # expanded_cols = melted_df['variable'].str.split(r'[_]', expand=True)
            expanded_cols = melted_df['variable'].str.split(r'__', expand=True)
            
            # Rename columns based on the maximum number of splits
            expanded_cols.columns = [f'variable_{i+1}' for i in range(expanded_cols.shape[1])]
            
            # Concatenate the expanded columns to the melted DataFrame
            melted_df = pd.concat([melted_df, expanded_cols], axis=1)
            
            # Rearrange columns: expanded columns after id_vars and before 'commission'
            id_vars = list(columns_before)
            value_name = 'commission'
            
            columns_order = id_vars + [col for col in expanded_cols.columns] + [value_name]
            melted_df = melted_df[columns_order]

                        # Remove rows where the commission value is a substring of any value in the id_vars columns
            rows_to_drop = melted_df.apply(
                lambda row: any(str(row[value_name]) in str(row[id_var]) for id_var in id_vars), axis=1)

            melted_df = melted_df[~rows_to_drop].reset_index(drop=True)

            melted_df.columns = [col.split('__')[-1] if col.lower().startswith('all broker') else col for col in melted_df.columns]

            melted_df= melted_df.dropna(subset=['commission']).reset_index(drop=True)

            melted_df.columns = rename_columns(melted_df)

            # Apply the function to the entire DataFrame
            melted_df = melted_df.applymap(replace_brackets)

            # Apply the function to the entire DataFrame
            melted_df = melted_df.applymap(strip_spaces)

            # Find all possible prefixes excluding certain prefixes
            all_prefixes = find_all_prefixes(melted_df.columns)
            # Remove all identified prefixes
            melted_df = remove_prefixes(melted_df, all_prefixes)
                        

else:
    # Initial headers are valid, use the initially read DataFrame with headers as the first row
    df = df_initial.copy()
    #Step 2: Drop rows with all NaN values
    df = df.dropna(how='all').reset_index(drop=True)

    # Drop columns where all values are NaN
    df = df.dropna(axis=1, how='all')

    # # Step 2: Initialize a list to store notes
    # notes = []
    # #print(notes)

    # # Step 3: Iterate through each row in the DataFrame to separate notes and remove them from df
    # index_to_drop = []  # To store indices of rows to drop from df
    # # for index, row in df.iterrows():
    # #     if pd.notna(row[0]) and not any(row.dropna().index[1:]):  # Check if it's a note row
    # #         notes.append(row[0])  # Append note to notes list
    # #         index_to_drop.append(index)  # Mark row index to drop

    # for index, row in df.iterrows():
    #     if (pd.notna(row[0]) or pd.notna(row[1])) and not any(row.dropna().index[2:]):  # Check if it's a note row
    #         notes.append(row[0] if pd.notna(row[0]) else row[1])  # Append note to notes list
    #         index_to_drop.append(index) 

    #     # Step 4: Keep rows up to and including the smallest index in index_to_drop
    # if index_to_drop:
    #     min_index = min(index_to_drop)  # Find the smallest index to keep
    #     df = df.iloc[:min_index].reset_index(drop=True)  # Keep rows from the beginning up to but not including this index
    
    new_df = df.copy()


    new_df = new_df.applymap(clean_text) ## for multiline text or wrap text in row or cell

            # Apply the cleaning function to the column names
    new_df.columns = [clean_text(col) for col in new_df.columns] ## removing text of multiline and wrap from heading

    new_df = clean_column_headers(new_df)

    # Call the function
    first_numerical_row_idx, columns_with_numerical_values = find_row_with_most_numerical_values(new_df)
    # columns_with_numerical_values, all_numerical_columns = find_all_numerical_columns(new_df)
            
    col_idx = new_df.columns.get_loc(columns_with_numerical_values)
    columns_before = new_df.columns[:col_idx]

    new_df[columns_before] = new_df[columns_before].fillna(method='ffill')

    last_numerical_row = find_last_numerical_row(new_df, columns_before)



    for col in new_df.columns[col_idx:]:
        new_df[col] = new_df[col].apply(lambda x: f"{float(x) * 100:.2f}" if isinstance(x, (int, float)) and str(x).startswith('0.') else x)  #% symbol removing and value at time of read convert to decimal reconvert to same


    # new_df = new_df.applymap(clean_text) ## for multiline text or wrap text in row or cell

    # #         # Apply the cleaning function to the column names
    # new_df.columns = [clean_text(col) for col in new_df.columns] ## removing text of multiline and wrap from heading

    # Loop through the columns and rename if they start with 'All_Broker_'
    new_columns = {col: col.split('__')[0] for col in df.columns if col.lower().startswith('all broker -')}

    # Rename the columns in the DataFrame
    new_df.rename(columns=new_columns, inplace=True)

    # melted_df= new_df.dropna(subset=[columns_with_numerical_values]).reset_index(drop=True)
    melted_df = new_df.copy()

    # Define a more flexible pattern for extracting details
    pattern = re.compile(r'Diesel upto (\d+)cc \(all model will cover\)\s*(?:and|&)?\s*all cars above (\d+)cc\s*(?:except|excluding)\s*(\w+)',re.IGNORECASE)

    # Initialize variables for extracted details
    diesel_cc, large_cars_cc, except_model = None, None, None

    # Search the entire DataFrame for the pattern in cell values
    for cell in melted_df.values.flatten():
        cell_str = str(cell)  # Convert cell to string for pattern matching
        match = pattern.search(cell_str)
        if match:
            diesel_cc, large_cars_cc, except_model = match.groups()
            # print(f"Found details - diesel_cc: {diesel_cc}, large_cars_cc: {large_cars_cc}, except_model: {except_model}")
            break

    # Define the replacement text if required values were found
    if diesel_cc and large_cars_cc and except_model:
        replacement_text = (f'All other Cars - Petrol upto {large_cars_cc}cc and Diesel above {diesel_cc}cc. Above {large_cars_cc}cc For {except_model}')
        # Replace the text across the entire DataFrame
        melted_df.replace('All other Cars except above model', replacement_text, inplace=True)

    # Define the substring to replace and the replacement value
    # substring_to_replace = 
    if except_model == 'TTMMHHK':
        replacement_value = 'TATA|TOYOTA|MARUTI|MAHINDRA|HONDA|HYUNDAI|KIA'

        # Replace the substring in the entire DataFrame
        melted_df = melted_df.applymap(lambda x: x.replace(except_model, replacement_value) if isinstance(x, str) else x)
    else:
        melted_df

    if columns_with_numerical_values == 'Renewal' or columns_with_numerical_values == 'commission':
        melted_df= melted_df.dropna(subset=[columns_with_numerical_values]).reset_index(drop=True)
        melted_df = melted_df.drop(columns= ['Comm on Net Premium of 1st year', 'Unnamed: 5'])

    else:
        melted_df


    melted_df.columns = rename_columns(melted_df)

    # Apply the function to the entire DataFrame
    melted_df = melted_df.applymap(replace_brackets)

    # # Remove rows where the commission value is a substring of any value in the id_vars columns
    # rows_to_drop = melted_df.apply(
    #     lambda row: any(str(row[value_name]) in str(row[id_var]) for id_var in id_vars), axis=1)

    # melted_df = melted_df[~rows_to_drop].reset_index(drop=True)

    # Apply the function to the entire DataFrame
    melted_df = melted_df.applymap(strip_spaces)

    # Find all possible prefixes excluding certain prefixes
    all_prefixes = find_all_prefixes(melted_df.columns)
    # Remove all identified prefixes
    melted_df = remove_prefixes(melted_df, all_prefixes)
    

In [42]:
table_start_idx
# columns

2

In [43]:
first_num_row_idx

1

In [44]:
df_initial.head(3)

,Product,OD Commission (including Rewards),TP Commission (including Rewards)
0,Private Car,20.0,15.0
1,Private Car Standalone OD,20.0,NaN
2,Private Car Standalone TP,NaN,15.0


In [45]:
df.head(2)

,Product,OD Commission (including Rewards),TP Commission (including Rewards)
0,Private Car,20.0,15.0
1,Private Car Standalone OD,20.0,NaN


In [46]:
new_df.head(2)

,Product,OD Commission (including Rewards),TP Commission (including Rewards)
0,Private Car,20.0,15.0
1,Private Car Standalone OD,20.0,NaN


In [47]:
melted_df.head(2)

,Product,OD Commission (including Rewards),TP Commission (including Rewards)
0,Private Car,20.0,15.0
1,Private Car Standalone OD,20.0,NaN


In [48]:
akhgklh

## Converting Json

In [49]:
# Function to find the first numerical column and its index
def find_first_numerical_column(df):
    for index, column in enumerate(df.columns):
        # Try converting the column to numeric
        numeric_column = pd.to_numeric(df[column], errors='coerce')
        # print(numeric_column)
        # Check if there are any non-NaN values
        if numeric_column.dropna().size > 0:
            if index == 0:
                continue
            return column, index
    return None, None

# def find_first_numerical_column(df):
#     for index, column in enumerate(df.columns[1:], start=1):  # Start from the second column
#         # Try converting the column to numeric
#         numeric_column = pd.to_numeric(df[column], errors='coerce')
#         # Check if there are any non-NaN values
#         if numeric_column.dropna().size > 0:
#             return column, index
#     return None, None


# Get the first numerical column name and index
first_numerical_column, column_index = find_first_numerical_column(melted_df)
print(f"First numerical column: {first_numerical_column}")
print(f"Index of the column: {column_index}")

First numerical column: OD Commission (including Rewards)
Index of the column: 1


In [50]:
import pandas as pd
import json
import numpy as np

def df_to_hierarchical_json(df, num_levels):
    """
    Convert a DataFrame to a hierarchical JSON format.

    Parameters:
        df (pd.DataFrame): The DataFrame to convert.
        num_levels (int): Number of hierarchical levels to extract from the start of the columns.

    Returns:
        str: JSON string of the hierarchical structure.
    """
    if num_levels > 1:
        # Ensure num_levels is not more than the number of columns
        num_levels = min(num_levels, len(df.columns))
        
        # Identify hierarchical columns
        level_columns = df.columns[:num_levels]
        print(level_columns)
        value_columns = df.columns[num_levels:]
        print(value_columns)
        
        # Initialize the hierarchical dictionary
        hierarchy = {}
        
        # Group by hierarchical levels
        grouped = df.groupby(list(level_columns))
        
        # Populate the hierarchical dictionary
        for keys, group in grouped:
            current_level = hierarchy
            for key in keys:
                key = str(key)
                if key not in current_level:
                    current_level[key] = {}
                current_level = current_level[key]
            
            # Append the values at the deepest level
            for _, row in group.iterrows():
                values = {col: (row[col] if not pd.isna(row[col]) else None) for col in value_columns}
                # current_level.setdefault('values', []).append(values)
                if 'values' not in current_level:
                    current_level['values'] = []
                current_level['values'].append(values)
        
        # Convert dictionary to JSON
        return json.dumps(hierarchy, indent=2)
    else:
            # Use columns before the numerical column as hierarchical levels
        # and columns after as value columns
        level_columns = df.columns[:column_index]
        value_columns = df.columns[column_index:]
        
        # Initialize the hierarchical dictionary
        hierarchy = {}

        # Populate the hierarchical dictionary
        for index, row in df.iterrows():
            current_level = hierarchy
            # Use the entire row for hierarchical levels
            for col in level_columns:
                key = row[col]
                key = str(key)
                if key not in current_level:
                    current_level[key] = {}
                current_level = current_level[key]

            # Append the values at the deepest level
            values = {col: row[col] if not pd.isna(row[col]) else None for col in value_columns}
            if 'values' not in current_level:
                current_level['values'] = []
            current_level['values'].append(values)

        # Convert dictionary to JSON
        return json.dumps(hierarchy, indent=2)

# Prompt the user for the number of levels
# num_levels = min(5, len(df.columns))


# Convert to hierarchical JSON with user-defined levels
hierarchical_json = df_to_hierarchical_json(melted_df, num_levels = column_index)

# Store the hierarchical JSON into a file
with open('New IndiaJuly.json', 'w') as file:
    file.write(hierarchical_json)


In [51]:
# melted_df['State (RTO)'].unique()

In [52]:
melted_df['variable_1'].unique()

KeyError: 'variable_1'

In [ ]:
# melted_df[(melted_df['State (RTO)']=='KA_Bangalore') & (melted_df['variable_3']== 'SCOOTER')]